<h1 style="font-size:3rem;color:orange;">Wordle Bot v.1.1</h1>

## Preliminaries

In [1]:
import matplotlib.pyplot as plt
from math import log
import time

In [2]:
f = open('allowed_words.txt','r')
ALLOWED_WORDS = list()
for line in f:
    line = line.rstrip()
    ALLOWED_WORDS.append(line)
f.close()

In [3]:
f = open('possible_answers.txt','r')
POSSIBLE_ANSWERS = list()
for line in f:
    line = line.rstrip()
    POSSIBLE_ANSWERS.append(line)
f.close()

## Mathematics

In [4]:
l = log(2)
def logBase2(n):
    """


    Parameters
    ----------
    n : int
        A number.

    Returns
    -------
    int
        Log base 2 of said number.

    """
    return log(n)/l

In [5]:
def convert_ternary(t):
    """


    Parameters
    ----------
    t : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.

    Returns
    -------
    int
        Base 10 representation of pattern.

    """
    return sum([t[i]*3**(4-i) for i in range(5)])

## Game mechanics

In [6]:
def get_feedback(guess,answer):
    """
    

    Parameters
    ----------
    guess : str
        Five-letter guess.
    answer : str
        Five-letter correct answer.

    Returns
    -------
    feedback : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.

    """
    #convert string to list
    temp = list(answer)
    answer = temp
    temp = list(guess)
    guess = temp
    
    #initialize
    feedback = ['']*5
    
    #isolate correctly placed letters
    for i in range(5):
        if guess[i] == answer[i]:
            feedback[i] = 2
            answer[i] = ''
            guess[i] = ''
    
    #isolate wrongly placed letters
    for i in range(5):
        if guess[i] == '': continue
        elif guess[i] in answer:
            feedback[i] = 1
            answer[answer.index(guess[i])] = ''
            guess[i] = ''
        else:
            feedback[i] = 0
    
    return feedback

## Entropy computation

In [7]:
def pattern_probability_distribution(allowed_words,guess):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.
    guess : str
        Five-letter guess.

    Returns
    -------
    pd : dict
        Contains the base 10 representation of a feedback pattern as the key.
        Corresponding value is its probability of appearing in the guess space.

    """
    total = len(allowed_words)
    pd = dict()
    for word in allowed_words:
        feedback = get_feedback(guess,word)
        feedback_enumerated = convert_ternary(feedback)
        pd[feedback_enumerated] = pd.get(feedback_enumerated,0) + 1/total
    return pd

In [8]:
def compute_entropy(allowed_words,guess):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.
    guess : str
        Five-letter guess.

    Returns
    -------
    res : float
        Expected entropy value of a guess, computed based on 
        pattern_probability_distribution.

    """
    pd = pattern_probability_distribution(allowed_words,guess)
    res = 0
    for p in pd.values():
        res += -p*logBase2(p)
    return round(res,2)

In [9]:
def compute_actual_entropy(allowed_words,guess,real_feedback):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.
    guess : str
        Five-letter guess.
    real_feedback : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.

    Returns
    -------
    updated_allowed_words : list
        Updates allowed_words by retaining only words fitting the actual feedback.

    """
    pd = pattern_probability_distribution(allowed_words,guess)
    p = pd[convert_ternary(real_feedback)]
    return round(-logBase2(p),2)

## WordleBot mechanics

### Reduce guess space

In [10]:
def reduce_allowed_words(allowed_words,guess,real_feedback):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.
    guess : str
        Five-letter guess.
    real_feedback : list
        Contains 05 elements, which can be 0, 1, or 2, denoting a feedback pattern.

    Returns
    -------
    updated_allowed_words : list
        Updates allowed_words by retaining only words fitting the actual feedback.

    """
    real_feedback_enumerated = convert_ternary(real_feedback)
    updated_allowed_words = list()
    for word in allowed_words:
        feedback_enumerated = convert_ternary(get_feedback(guess,word))
        if feedback_enumerated == real_feedback_enumerated:
            updated_allowed_words.append(word)
    
    return updated_allowed_words

### Entropy ranker

In [11]:
def get_ranker(allowed_words):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.

    Returns
    -------
    ranker : list
        Contains ~13000 tuples, whose first element is a guess, and
        second element is the entropy of that guess.

    """
    ranker = list()
    for word in allowed_words:
        ranker.append((word,compute_entropy(allowed_words,word)))
    ranker.sort(key = lambda t: t[1], reverse = True)
    
    return ranker

### Interface

In [12]:
def display_ranker(allowed_words):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.

    Returns
    -------
    None.
    Prints the ranker.

    """
    ranker = get_ranker(allowed_words)
    print('{0:<10}{1:<10}'.format('Word','Expected entropy'))
    for (word,entropy) in ranker[:10]: #print only top ten words with highest entropy
        print('{0:<10}{1:<10.2f}'.format(word,entropy))

In [13]:
def check_win(feedback):
    """
    

    Parameters
    ----------
    feedback : list
        Contains 05 elements, which can be 0, 1, or 2.

    Returns
    -------
    win : bool
        Becomes True when feedback is a list of 05 2's.

    """
    win = True
    for i in range(5):
        if feedback[i] != 2: 
            win = False
            break
    return win

In [14]:
def wordlebot_interface(allowed_words):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.

    Returns
    -------
    None.
    Prints the interactive program for user to play Wordle and input real feedback.

    """
    win = False
    valid_words = allowed_words
    i = 0
    while not win:
        print("Guess #" + str(i+1))
        if i == 0: 
            pass #skip entropy computation for first guess - dev purpose
        else:
            display_ranker(valid_words)
        guess = input('> Enter your guess: ')
        real_feedback = list(map(int,input('>> Enter the feedback: ').split(' ')))
        
        
        if check_win(real_feedback) == True:
            print(">>> Complete!")
            break
        print(">>> Expected entropy: " + str(compute_entropy(valid_words,guess)))
        print("    Actual entropy: " + str(compute_actual_entropy(valid_words,guess,real_feedback)))
        
        temp = reduce_allowed_words(valid_words,guess,real_feedback)
        valid_words = temp
        print(">>>> Remaining possibilities: " + str(len(valid_words)) + "\n")
        
        i += 1

## Performance testing

In [15]:
def wordlebot_play(allowed_words,answer):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains allowed guesses.
    answer : str
        Five-letter actual answer.

    Returns
    -------
    guess_count : int
        Number of guesses needed to reach the actual answer.

    """
    win = False
    valid_words = allowed_words
    i = 0
    guess_count = 0
    
    while not win:
        if i == 0: 
            pass #skip entropy computation for first guess - dev purpose
        else:
            ranker = get_ranker(valid_words)
        
        if i == 0:
            guess = 'tares'
        else:
            guess = ranker[0][0]
        guess_count += 1
        
        real_feedback = get_feedback(guess,answer)
        
        if check_win(real_feedback) == True:
            break
        
        temp = reduce_allowed_words(valid_words,guess,real_feedback)
        valid_words = temp
        i += 1
    
    return guess_count

In [16]:
def test_for_performance(possible_answers,n="all"):
    """
    

    Parameters
    ----------
    possible_answers : list
        Contains ~2300 human-curated possible answers.
    n : int
        Default to "all" - if n is not given, test on all possible answers.
        If n is given, test on first n answers of all possible answers.

    Returns
    -------
    None.
    Prints bar plot showing frequency of number of guesses needed.

    """
    #initialize
    performance_count = dict()
    
    #gameplay for ~2300 words in POSSIBLE_ANSWERS
    if n == "all":
        for word in possible_answers:
            guess_count = wordlebot_play(ALLOWED_WORDS,word)
            performance_count[guess_count] = performance_count.get(guess_count,0) + 1
    else:
        for word in possible_answers[:n]:
            guess_count = wordlebot_play(ALLOWED_WORDS,word)
            performance_count[guess_count] = performance_count.get(guess_count,0) + 1
    
    #visualize
    x = list(range(1,max(performance_count.keys())+1))
    y = [performance_count.get(i,0) for i in x]
    plt.bar(x,y,color='royalblue',alpha=0.7)
    plt.grid(color='#95a5a6', linestyle='--', linewidth=1, axis='y', alpha=0.7)
    plt.title('WordleBot - Test performance')
    plt.xlabel('Number of guesses needed')
    plt.ylabel('Frequency')
    plt.show()

In [17]:
def test_for_time_complexity(allowed_words):
    """
    

    Parameters
    ----------
    allowed_words : list
        Contains valid guesses.

    Returns
    -------
    None.
    Prints line graph showing time complexity based on the number of words whose
    entropy is to be calculated and ranked.

    """

    #initialize
    time_complexities = list()
    interval = [10,30,100,300,1000,3000]
    
    for n in interval:
        ranker = list()
        start = time.time()
        for word in allowed_words[:n]:
            ranker.append((word,compute_entropy(allowed_words,word)))
        ranker.sort(key = lambda t: t[1], reverse = True)
        end = time.time()
        time_complexities.append(end-start)
    
    #visualize    
    plt.plot(interval,time_complexities)
    plt.title('WordleBot - Time complexities')
    plt.xlabel('Number of considered words')
    plt.ylabel('Time (s)')
    plt.show()